<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Data_Automation_and_Analysis_(ETL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import logging
import os

logging.basicConfig(level=logging.INFO)

def extract_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        raise ValueError(f"API request failed: {e}")
    except ValueError as e:
        raise ValueError(f"Failed to parse JSON: {e}")

def validate_data(df):
    if 'date' not in df.columns or 'value' not in df.columns:
        raise ValueError("Dataframe must contain 'date' and 'value' columns.")

def transform_data(df, fill_value=0):
    validate_data(df)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['cleaned_value'] = df['value'].fillna(fill_value).astype(float)
    return df[['date', 'cleaned_value']]

def load_data(df, db_connection_string, table_name):
    engine = create_engine(db_connection_string)
    df.to_sql(table_name, con=engine, if_exists='append', index=False)

def etl_pipeline(api_url, db_connection_string, table_name):
    logging.info("Starting ETL pipeline")
    try:
        df = extract_data(api_url)
        df_transformed = transform_data(df)
        load_data(df_transformed, db_connection_string, table_name)
        logging.info("ETL pipeline completed successfully")
    except Exception as e:
        logging.error(f"ETL pipeline failed: {e}")

# Replace with your actual API URL and table name
api_url = "https://api.example.com/data"
db_connection_string = os.getenv("DB_CONNECTION_STRING", "sqlite:///my_database.db")
table_name = "my_table"

etl_pipeline(api_url, db_connection_string, table_name)